# Workshop-Notebook (Google Colab): Manuell vs. KI (BirdNET)
Sehr einfache Schritte. Bitte **Zelle für Zelle** ausführen.

## Was dieses Notebook macht
1. (Optional) GitHub-Repo klonen
2. Dateien holen (Audio + Artenliste)
3. BirdNET installieren und Modell laden
4. BirdNET laufen lassen
5. CSV speichern + einfache Plots
6. Gruppe A und B vergleichen

---
### Wichtiger Hinweis
Deine Audio-Dateien sind oft **nicht** im GitHub-Repo (weil groß). Dann musst du sie **hochladen** oder aus **Google Drive** laden.


## 0) Colab-Check + (optional) Google Drive verbinden
Wenn du Drive nutzt: zuerst diese Zelle ausführen.

In [ ]:
IN_COLAB = False
try:
    import google.colab  # type: ignore
    IN_COLAB = True
except Exception:
    IN_COLAB = False

print('IN_COLAB =', IN_COLAB)

# Optional: Drive mounten (nur in Colab)
if IN_COLAB:
    from google.colab import drive  # type: ignore
    drive.mount('/content/drive')


## 1) (Optional) GitHub-Repo klonen
Wenn du **kein Repo** nutzen willst: lass `REPO_URL` leer und überspringe diese Zelle.

In [ ]:
import os
from pathlib import Path

# --- HIER EINTRAGEN (oder leer lassen) ---
REPO_URL = ""  # z.B. "https://github.com/DEIN_USERNAME/workshop-manual-vs-ki-birdnet.git"
REPO_FOLDER = "workshop-repo"

if REPO_URL.strip():
    if not Path(REPO_FOLDER).exists():
        !git clone {REPO_URL} {REPO_FOLDER}
    %cd {REPO_FOLDER}
    print('Ordnerinhalt:')
    !ls -la
else:
    print('Kein Repo geklont (REPO_URL ist leer).')


## 2) Pakete installieren
Diese Zelle einmal ausführen.

In [ ]:
%pip -q install birdnet pandas numpy matplotlib librosa soundfile ipywidgets


## 3) Imports + Helfer
Diese Zelle ausführen.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import birdnet
import librosa
import librosa.display

from pathlib import Path

def show_files_here():
    print('Dateien im aktuellen Ordner:')
    for p in sorted(Path('.').glob('*')):
        print(' -', p)

def ensure_path(p: str) -> Path:
    pp = Path(p).expanduser()
    if not pp.exists():
        raise FileNotFoundError(f"Nicht gefunden: {pp}")
    return pp

def species_split(species_name: str):
    # BirdNET nutzt oft: "Genus species_Common Name"
    if '_' in species_name:
        sci, common = species_name.split('_', 1)
        return sci.strip(), common.strip()
    return species_name.strip(), ''

def time_to_seconds(t):
    parts = str(t).split(':')
    parts = [float(x) for x in parts]
    if len(parts) == 3:
        return parts[0]*3600 + parts[1]*60 + parts[2]
    if len(parts) == 2:
        return parts[0]*60 + parts[1]
    return float(parts[0])


## 4) Dateien bereitstellen (Audio + Artenliste)
Du hast 2 Wege:
1. **Upload** in Colab (einfach) oder
2. **Google Drive** Pfade setzen

### 4A) Upload (empfohlen für Workshop)
Führe die nächste Zelle aus und lade hoch:
- `GroupA_mix.wav`
- `GroupB_mix.wav`
- optional: `Arten_NRW.txt`


In [ ]:
if IN_COLAB:
    from google.colab import files  # type: ignore
    uploaded = files.upload()
    print('Upload fertig.')
else:
    print('Nicht in Colab -> Upload überspringen.')
show_files_here()


## 5) Pfade setzen
Wenn du Upload gemacht hast, reichen meistens diese Namen.
Wenn du Drive nutzt, setze komplette Pfade.

In [ ]:
# --- STANDARD (Upload) ---
GROUP_A_AUDIO = 'GroupA_mix.wav'
GROUP_B_AUDIO = 'GroupB_mix.wav'

# Optional: Artenliste (eine Art pro Zeile). Falls nicht vorhanden: None setzen.
NRW_SPECIES_LIST = 'Arten_NRW.txt'
# NRW_SPECIES_LIST = None

OUT_DIR = Path('./workshop_outputs')
OUT_DIR.mkdir(exist_ok=True, parents=True)

# Prüfen
a_path = ensure_path(GROUP_A_AUDIO)
b_path = ensure_path(GROUP_B_AUDIO)

species_list_path = None
if NRW_SPECIES_LIST is not None:
    p = Path(NRW_SPECIES_LIST)
    if p.exists():
        species_list_path = str(p)
    else:
        print('Artenliste nicht gefunden -> ohne Filter')
        species_list_path = None

print('Group A:', a_path)
print('Group B:', b_path)
print('Species list:', species_list_path)


## 6) BirdNET Modell laden
Beim ersten Mal lädt BirdNET das Modell automatisch.

In [ ]:
model = birdnet.load('acoustic', '2.4', 'tf')
print('Modell geladen.')


## 7) BirdNET laufen lassen (Gruppe A + Gruppe B)
Ergebnis: Tabelle mit `confidence`.

In [ ]:
def run_birdnet(audio_path: Path, label: str, custom_species_list: str | None):
    preds = model.predict(str(audio_path), custom_species_list=custom_species_list)
    preds = preds.copy()
    preds['file_label'] = label
    preds['start_sec'] = preds['start_time'].apply(time_to_seconds)
    preds['end_sec'] = preds['end_time'].apply(time_to_seconds)
    preds[['scientific', 'common']] = preds['species_name'].apply(lambda s: pd.Series(species_split(s)))
    return preds

preds_a = run_birdnet(a_path, 'GruppeA', species_list_path)
preds_b = run_birdnet(b_path, 'GruppeB', species_list_path)
preds_all = pd.concat([preds_a, preds_b], ignore_index=True)

preds_all.head()


## 8) CSV speichern
Wir speichern 3 CSV-Dateien.

In [ ]:
out_all = OUT_DIR / 'birdnet_predictions_all.csv'
out_a   = OUT_DIR / 'birdnet_predictions_GroupA.csv'
out_b   = OUT_DIR / 'birdnet_predictions_GroupB.csv'

preds_all.to_csv(out_all, index=False)
preds_a.to_csv(out_a, index=False)
preds_b.to_csv(out_b, index=False)

print('Gespeichert:', out_all)
print('Gespeichert:', out_a)
print('Gespeichert:', out_b)


## 9) Plots
### 9.1 Top-Arten (max Confidence)

In [ ]:
def top_species_plot(df: pd.DataFrame, title: str, top_n: int = 12):
    g = (df.groupby('species_name')['confidence']
           .max()
           .sort_values(ascending=False)
           .head(top_n))
    plt.figure()
    g.sort_values().plot(kind='barh')
    plt.title(title)
    plt.xlabel('Max Confidence')
    plt.tight_layout()
    plt.show()

top_species_plot(preds_a, 'Gruppe A – Top-Arten')
top_species_plot(preds_b, 'Gruppe B – Top-Arten')


### 9.2 Confidence-Histogramm

In [ ]:
def confidence_hist(df: pd.DataFrame, title: str):
    plt.figure()
    plt.hist(df['confidence'].values, bins=30)
    plt.title(title)
    plt.xlabel('Confidence')
    plt.ylabel('Anzahl')
    plt.tight_layout()
    plt.show()

confidence_hist(preds_a, 'Gruppe A – Confidence')
confidence_hist(preds_b, 'Gruppe B – Confidence')


## 10) Vergleich A vs. B
Welche Arten sind nur in A, nur in B, oder in beiden?

In [ ]:
THRESH = 0.25  # im Workshop ändern

set_a = set(preds_a.loc[preds_a['confidence'] >= THRESH, 'species_name'].unique())
set_b = set(preds_b.loc[preds_b['confidence'] >= THRESH, 'species_name'].unique())

only_a = sorted(set_a - set_b)
only_b = sorted(set_b - set_a)
both   = sorted(set_a & set_b)

print('Schwelle:', THRESH)
print('Nur in A:', len(only_a))
print('Nur in B:', len(only_b))
print('In beiden:', len(both))

pd.DataFrame({'species_name': only_a}).to_csv(OUT_DIR / 'only_in_GroupA.csv', index=False)
pd.DataFrame({'species_name': only_b}).to_csv(OUT_DIR / 'only_in_GroupB.csv', index=False)
pd.DataFrame({'species_name': both}).to_csv(OUT_DIR / 'in_both.csv', index=False)


## 11) Optional: Clip-Nummern (wenn Mix = 8×25s + 1s Pause)
Dann bekommst du pro Clip die beste Vorhersage.

In [ ]:
CLIP_LEN = 25
GAP_LEN = 1
BLOCK = CLIP_LEN + GAP_LEN

def add_clip_index(df: pd.DataFrame):
    df = df.copy()
    df['clip_idx'] = (df['start_sec'] // BLOCK).astype(int) + 1
    within_clip = (df['start_sec'] % BLOCK) < CLIP_LEN
    return df[within_clip].copy()

preds_a_clip = add_clip_index(preds_a)
preds_b_clip = add_clip_index(preds_b)

top_a_by_clip = (preds_a_clip.sort_values('confidence', ascending=False)
                 .groupby('clip_idx', as_index=False)
                 .first()[['clip_idx','species_name','confidence','scientific','common']])

top_b_by_clip = (preds_b_clip.sort_values('confidence', ascending=False)
                 .groupby('clip_idx', as_index=False)
                 .first()[['clip_idx','species_name','confidence','scientific','common']])

top_a_by_clip.to_csv(OUT_DIR / 'GroupA_top_prediction_per_clip.csv', index=False)
top_b_by_clip.to_csv(OUT_DIR / 'GroupB_top_prediction_per_clip.csv', index=False)

top_a_by_clip, top_b_by_clip


## 12) Optional: Sonogramm (Spektrogramm) zum Üben
Wähle ein Zeitfenster und zeichne es.

In [ ]:
AUDIO_FOR_SONOGRAM = a_path  # oder b_path
START_SEC = 0
DURATION_SEC = 12

y, sr = librosa.load(str(AUDIO_FOR_SONOGRAM), sr=None, offset=START_SEC, duration=DURATION_SEC)
S = librosa.stft(y, n_fft=2048, hop_length=256)
S_db = librosa.amplitude_to_db(np.abs(S), ref=np.max)

plt.figure(figsize=(10, 4))
librosa.display.specshow(S_db, sr=sr, hop_length=256, x_axis='time', y_axis='hz')
plt.title(f'Spektrogramm: {AUDIO_FOR_SONOGRAM.name} (t={START_SEC}s..{START_SEC+DURATION_SEC}s)')
plt.colorbar(format='%+2.0f dB')
plt.tight_layout()
plt.show()


## 13) Optional: Ergebnisse als ZIP herunterladen

In [ ]:
import shutil
zip_path = shutil.make_archive('workshop_outputs', 'zip', root_dir=str(OUT_DIR))
print('ZIP erstellt:', zip_path)

if IN_COLAB:
    from google.colab import files  # type: ignore
    files.download(zip_path)
